## 연습문제 1

다음 4건의 데이터에 대해 Merkle Root 값을 계산하고 출력하세요.

중간 노드의 AB, CD의 해시도 출력하세요.

* txA = 'Hello'

* txB = 'How are you?'

* txC = 'This is Thursday'

* txD = 'Happy new Year'

In [6]:
import hashlib

txA = 'Hello'.encode()
txB = 'How are you?'.encode()
txC = 'This is Thursday'.encode()
txD = 'Happy new Year'.encode()

In [7]:
# hash 값
hashA = hashlib.sha256(txA)
hashB = hashlib.sha256(txB)
hashC = hashlib.sha256(txC)
hashD = hashlib.sha256(txD)

In [12]:
# byteswap
hashAswap = "".join(reversed([hashA.hexdigest()[i:i+2] for i in range(0, hashA.digest_size*2, 2)]))
hashBswap = "".join(reversed([hashB.hexdigest()[i:i+2] for i in range(0, hashB.digest_size*2, 2)]))
hashCswap = "".join(reversed([hashC.hexdigest()[i:i+2] for i in range(0, hashC.digest_size*2, 2)]))
hashDswap = "".join(reversed([hashD.hexdigest()[i:i+2] for i in range(0, hashD.digest_size*2, 2)]))

In [19]:
# double hashing
hashAB = hashAswap + hashBswap
doubleHashAB = hashlib.sha256(hashAB.encode()).hexdigest()
hashCD = hashCswap + hashDswap
doubleHashCD = hashlib.sha256(hashCD.encode()).hexdigest()

print("중간 노드 AB의 해시: ", doubleHashAB, "\n중간 노드 CD의 해시: ", doubleHashCD)

중간 노드 AB의 해시:  a7dca36b1e6fe56e1d7bc34fe56ceea8ab64f47a55e929b712fab58885dcdc37 
중간 노드 CD의 해시:  15524f5c1b536c2fe0f717fd8b1bbede37f94654fae6f47c20521ce476c8e0b2


In [22]:
# Merkle Root의 값을 계산하기 위해 위와 같은 작업을 반복 실행
# AB와 CD의 해시값은 위와 같이 구했으므로 byteswap부터 실행
merkleABswap = "".join(reversed([doubleHashAB[i:i+2] for i in range(0, len(doubleHashAB), 2)]))
merkleCDswap = "".join(reversed([doubleHashCD[i:i+2] for i in range(0, len(doubleHashCD), 2)]))

In [23]:
# double hashing
merkleABCD = merkleABswap + merkleCDswap
doubleMerkle = hashlib.sha256(merkleABCD.encode()).hexdigest()

print("Merkle Root의 값: ", doubleMerkle)

Merkle Root의 값:  00aa61af726d4cd0d5a5e0d2e3e3280552acb6d587c15413a713b7a96275cefc


## 연습문제 2

친구에게  주소를 구하고, 그 주소로 송금해 보자. 송금이 되지 않으면 왜 안되는지 이유를 알아보자.

안되면 친구의 주소를 만들고 전송한다.

잔고의 증가분을 출력하세요.

소요된 gas비용 출력하세요.

In [38]:
%%writefile ../src/ethTrans.js
var peer = "0x77d201694d9d1c3fbd44a56dab324ef299bc3fb1"
var bal1 = eth.getBalance(eth.coinbase);
var bal2 = eth.getBalance(peer);
console.log('my account balance in ether: ', web3.fromWei(bal1, "ether"));
console.log('peer balance in ether: ', web3.fromWei(bal2, "ether"));
console.log('block number: ', eth.blockNumber);
console.log('transaction count: ', eth.getTransactionCount(eth.coinbase));

var h = eth.sendTransaction({from:eth.coinbase, to:peer, value:10000});
console.log("...mining start");
miner.start();admin.sleepBlocks(1);miner.stop();
console.log("mining done...");
var bal1new = eth.getBalance(eth.coinbase);
var bal2new = eth.getBalance(peer);
console.log('my account balance in ether: ', web3.fromWei(bal1new, "ether"));
console.log('peer balance in ether: ', web3.fromWei(bal2new, "ether"));
console.log('block number: ', eth.blockNumber);
console.log('transaction count: ', eth.getTransactionCount(eth.coinbase));

console.log("\n\nincreased balance: ", bal2new - bal2);
console.log("gasUsed: ", eth.getTransactionReceipt(h).gasUsed);

Overwriting ../src/ethTrans.js


In [39]:
!geth --exec "loadScript('../src/ethTrans.js')" attach http://localhost:8445

my account balance in ether:  534.99999999999987
peer balance in ether:  1.2e-13
block number:  113
transaction count:  13
...mining start
mining done...
my account balance in ether:  544.99999999999986
peer balance in ether:  1.3e-13
block number:  115
transaction count:  14


increased balance:  10000
gasUsed:  21000
true


## 연습문제 3

해시는 100미만의 양수로 정해진다고 하자.

NONCE는 반복회수로만 쓰이고 무작위 수를 생성하는데 입력되지는 않는다.

목표해시는 난이도에 따라 결정이 된다고 하자.

마이닝을 해서, 목표 해시를 찾아보자.

- 90을 목표해시로 정하고 몇 회만에 마이닝에 성공하는지 출력

- 10을 목표해시로 정하고 몇 회만에 마이닝에 성공하는지 출력



In [47]:
import hashlib
import random

found = False
NONCE = 0

while found == False:
    guessHash = random.randint(1, 99)
    if (guessHash < 90):
        found = True
    NONCE += 1
    print("NONCE: ", NONCE, "guessHash: ", guessHash)
print("Solved NONCE: ", NONCE, "guessHash: ", guessHash)
print("90을 목표 해시로 정했을 경우", NONCE, "회 만에 마이닝에 성공")

NONCE:  1 guessHash:  55
Solved NONCE:  1 guessHash:  55
90을 목표 해시로 정했을 경우 1 회 만에 마이닝에 성공


In [48]:
found = False
NONCE = 0

while found == False:
    guessHash = random.randint(1, 99)
    if (guessHash < 10):
        found = True
    NONCE += 1
    print("NONCE: ", NONCE, "guessHash: ", guessHash)
print("Solved NONCE: ", NONCE, "guessHash: ", guessHash)
print("10을 목표 해시로 정했을 경우", NONCE, "회 만에 마이닝에 성공")

NONCE:  1 guessHash:  68
NONCE:  2 guessHash:  24
NONCE:  3 guessHash:  35
NONCE:  4 guessHash:  27
NONCE:  5 guessHash:  45
NONCE:  6 guessHash:  3
Solved NONCE:  6 guessHash:  3
10을 목표 해시로 정했을 경우 6 회 만에 마이닝에 성공
